# Ejemplo básico aplicado de producto cartesiano, relación y conjuntos

## Producto Cartesiano y Relación

Para aquellos que utilicen SQL, la *query* (consulta)
```
SELECT *
FROM Tabla1, Tabla2;
```
recupera todos los elementos de $Tabla1 \times Tabla2$, debido a que la coma `,` representa el producto cartesiano. 

### Imports, etc.

In [ ]:
% pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=f7c5d8662a088bde53ed33ce9518b4b34fe8c3b4c4840ffa2221bb368ca5d357
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [ ]:
import pandas as pd
import pandasql as ps

# helper
q = lambda s : ps.sqldf(s, globals())

# levantamos los datasets
compras = pd.read_csv("data/cosas_a_comprar.csv", header=0)
invitados = pd.read_csv("data/invitados_juntada.csv", header=0)

### ¿Quién trae qué?
Invitados a una reunión tienen que organizarse para ver quién trae cada cosa. Para eso cuentan con los siguientes datos:
* Una tabla de *compras* con cada item de la lista de cosas a comprar y su precio
* Una tabla de *invitados* con la lista de invitados y, para cada uno, cuánto dinero puede disponer para comprar lo suyo.

In [ ]:
# tabla de cosas a comprar
q("""
SELECT *
FROM compras;
""")

,Cosa,Precio
0,bebida,200
1,papas fritas,500
2,hamburguesas,1500


In [ ]:
# tabla de invitados
q("""
SELECT *
FROM invitados;
""")

,Invitado,Disponible
0,Martin,400
1,Vero,600
2,Magui,800
3,Julian,50
4,Jemi,1500


In [ ]:
# producto cartesiano de invitados x compras
q("""
SELECT *
FROM invitados, compras;
""")

,Invitado,Disponible,Cosa,Precio
0,Martin,400,bebida,200
1,Martin,400,papas fritas,500
2,Martin,400,hamburguesas,1500
3,Vero,600,bebida,200
4,Vero,600,papas fritas,500
5,Vero,600,hamburguesas,1500
6,Magui,800,bebida,200
7,Magui,800,papas fritas,500
8,Magui,800,hamburguesas,1500
9,Julian,50,bebida,200


Observemos que el producto cartesiano de $compras \times invitados$ NO es el mismo que $invitados \times compras$.

In [ ]:
# producto cartesiano de compras x invitados
q("""
SELECT *
FROM compras,invitados;
""")

,Cosa,Precio,Invitado,Disponible
0,bebida,200,Martin,400
1,bebida,200,Vero,600
2,bebida,200,Magui,800
3,bebida,200,Julian,50
4,bebida,200,Jemi,1500
5,papas fritas,500,Martin,400
6,papas fritas,500,Vero,600
7,papas fritas,500,Magui,800
8,papas fritas,500,Julian,50
9,papas fritas,500,Jemi,1500


Nos interesa considerar a los invitados que ya sabemos que pueden comprar lo que les pedimos, entonces en realidad queremos un *subconjunto del conjunto cartesiano que cumple cierta condición*... es decir queremos los elementos de la relación $\mathcal{R}$ donde: 
$$
invitado \; \mathcal{R} \; compra \Leftrightarrow invitado.Disponible \geq compra.Precio
$$

Esto en SQL es un INNER JOIN (o simplemente JOIN):
```
SELECT *
FROM Tabla1 JOIN Tabla2
ON condicion;
```

In [ ]:
q("""
SELECT * 
FROM invitados JOIN compras
ON invitados.Disponible >= compras.Precio;
""")

,Invitado,Disponible,Cosa,Precio
0,Martin,400,bebida,200
1,Vero,600,bebida,200
2,Vero,600,papas fritas,500
3,Magui,800,bebida,200
4,Magui,800,papas fritas,500
5,Jemi,1500,bebida,200
6,Jemi,1500,papas fritas,500
7,Jemi,1500,hamburguesas,1500


## Conjuntos

Python tiene soporte nativo para conjuntos via `set` y `frozenset`, siendo su única diferencia que los `frozenset` son inmutables.

¿Por qué alguien querría una estructura inmutable?
* Para trabajar con un paradigma tipo funcional donde los elementos no se modifican, se hacen nuevos
* Para utilizarla como clave en un diccionario (las claves en diccionarios deben ser hasheables, y las estructuras inmutables son hasheables)

Si no vamos a hacer ninguna de las dos cosas, probablemente no nos importe la diferencia y los usemos de forma indistinta

In [ ]:
# definimos algunos conjuntos
primos = {2,3,5,7,11}
pares_menores_que_20 = {num for num in range(1,21) if num % 2 == 0} # es como una list comprehension!
algunos_pares = set([2,6,10])

type(primos), type(pares_menores_que_20), type(algunos_pares)

(set, set, set)

In [ ]:
# operacion de pertenencia
3 in primos

True

In [ ]:
# operacion de interseccion
primos & algunos_pares

{2}

In [ ]:
# operacion de union
primos | algunos_pares

{2, 3, 5, 6, 7, 10, 11}

In [ ]:
# operacion de diferencia
primos - algunos_pares

{3, 5, 7, 11}

El producto cartesiano está disponible en la librería estándar *itertools* bajo el nombre de `product()`. Aún así, no es difícil reproducirlo sin la misma. La diferencia fundamental es que `product()` devuelve un iterable, con lo cual está en forma "implícita" (hay que recorrerlo, o pasarlo a un nuevo conjunto).

In [ ]:
# con itertools
from itertools import product

product(primos, algunos_pares)

In [ ]:
# si lo pasamos a un set
set(product(primos, algunos_pares))

{(2, 2),
 (2, 6),
 (2, 10),
 (3, 2),
 (3, 6),
 (3, 10),
 (5, 2),
 (5, 6),
 (5, 10),
 (7, 2),
 (7, 6),
 (7, 10),
 (11, 2),
 (11, 6),
 (11, 10)}

In [ ]:
# a mano
{(x,y) for x in primos for y in algunos_pares}

{(2, 2),
 (2, 6),
 (2, 10),
 (3, 2),
 (3, 6),
 (3, 10),
 (5, 2),
 (5, 6),
 (5, 10),
 (7, 2),
 (7, 6),
 (7, 10),
 (11, 2),
 (11, 6),
 (11, 10)}

Python establece que los conjuntos se ordenan en forma parcial, donde dos conjuntos $A, B$ son tales que $A \leq B \Leftrightarrow A \subseteq B$, es decir si $A$ está contenido en $B$. Así, el testeo por contención (estricta y no) se puede hacer con las simples comparaciones de orden.

In [ ]:
# algunos_pares es subconjunto estricto de pares_menores_que_20 ?
algunos_pares < pares_menores_que_20

True

In [ ]:
# un conjunto está estrictamente contenido en si mismo?
primos < primos

False

In [ ]:
# y sólo contenido?
primos <= primos

True

In [ ]:
# dos conjuntos son iguales si tienen exactamente los mismos elementos
{3,4,5} == {5,4,3}

True